## Find out the users that post the most in a set of subreddits

edit the `sub_names` var to add/remove subreddits from the calculation

This one is for the month

In [ ]:
import praw
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

##### vars #####
show = 20
time_filter = "month"
sub_names = ['politics', 'all', 'news', 'interestingasfuck', 'whitePeopleTwitter', 'worldnews', 'todayilearned', 'BlackPeopleTwitter', 'PoliticalHumor', 'MadeMeSmile', 'memes', 'pics', 'antiwork', 'PublicFreakout', 'funny', 'askreddit', 'maybemaybemaybe', 'oddlysatisfying', 'nextfuckinglevel', 'science']
################

reddit = praw.Reddit(
    client_id=os.getenv('client_id'),
    client_secret=os.getenv('client_secret'),
    user_agent="submissions comments",
    username=os.getenv('username'),
)

def serialize_reddit(s): 
    return {
        "author": s.author,
        "created_utc": s.created_utc,
    }
def flatten(l):
    return [item for sublist in l for item in sublist]


submissions = list(flatten(reddit.subreddit(s).top(time_filter=time_filter,limit=200) for s in sub_names))
df = pd.DataFrame([serialize_reddit(s) for s in submissions])
df['created_utc'] = pd.to_datetime(df.created_utc.values, unit='s', utc=True)

print(sub_names)

ndf = df.query('author != "None"')
print('Total submissions:', len(ndf))

print('Number of unique authors:', ndf.author.nunique())

csum = ndf.author.value_counts(1).cumsum()

bins = pd.cut(csum, [0., 0.25, 0.5, 0.75, 1.], labels=['25%','50%','75%','100%'])

print('\nQuartiles:')
print(csum.groupby(bins).size())

print(f'\nTop {show} authors:')
topa = pd.concat([ndf.author.value_counts().head(show), ndf.author.value_counts(1).head(show)*100],axis=1)
topa.index.name = 'Author'
topa.columns = ('Count', 'Percent')
print(topa)

print(f'\nPercent sum {ndf.author.value_counts(1).head(show).sum() * 100}')
print('\n======\n')

## Find out the users that post the most in a set of subreddits

edit the `sub_names` var to add/remove subreddits from the calculation

This one is for the year

In [12]:
import praw
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

##### vars #####
show = 20
time_filter = "year"
sub_names = ['politics', 'all', 'news', 'interestingasfuck', 'whitePeopleTwitter', 'worldnews', 'todayilearned', 'BlackPeopleTwitter', 'PoliticalHumor', 'MadeMeSmile', 'memes', 'pics', 'antiwork', 'PublicFreakout', 'funny', 'askreddit', 'maybemaybemaybe', 'oddlysatisfying', 'nextfuckinglevel', 'science']
################

reddit = praw.Reddit(
    client_id=os.getenv('client_id'),
    client_secret=os.getenv('client_secret'),
    user_agent="submissions comments",
    username=os.getenv('username'),
)

def serialize_reddit(s): 
    return {
        "author": s.author,
        "created_utc": s.created_utc,
    }
def flatten(l):
    return [item for sublist in l for item in sublist]


submissions = list(flatten(reddit.subreddit(s).top(time_filter=time_filter,limit=200) for s in sub_names))
df = pd.DataFrame([serialize_reddit(s) for s in submissions])
df['created_utc'] = pd.to_datetime(df.created_utc.values, unit='s', utc=True)

print(sub_names)

ndf = df.query('author != "None"')
print('Total submissions:', len(ndf))

print('Number of unique authors:', ndf.author.nunique())

csum = ndf.author.value_counts(1).cumsum()

bins = pd.cut(csum, [0., 0.25, 0.5, 0.75, 1.], labels=['25%','50%','75%','100%'])

print('\nQuartiles:')
print(csum.groupby(bins).size())

print(f'\nTop {show} authors:')
topa = pd.concat([ndf.author.value_counts().head(show), ndf.author.value_counts(1).head(show)*100],axis=1)
topa.index.name = 'Author'
topa.columns = ('Count', 'Percent')
print(topa)

print(f'\nPercent sum {ndf.author.value_counts(1).head(show).sum() * 100}')
print('\n======\n')

['politics', 'all', 'news', 'interestingasfuck', 'whitePeopleTwitter', 'worldnews', 'todayilearned', 'BlackPeopleTwitter', 'PoliticalHumor', 'MadeMeSmile', 'memes', 'pics', 'antiwork', 'PublicFreakout', 'funny', 'askreddit', 'maybemaybemaybe', 'oddlysatisfying', 'nextfuckinglevel', 'science']
Total submissions: 4000
Number of unique authors: 2689

Quartiles:
author
25%     152
50%     593
75%     972
100%    971
Name: author, dtype: int64

Top 20 authors:
                      Count   Percent
Author                               
DaFunkJunkie             63  1.620787
Wagamaga                 29  0.746077
PeasKhichra              20  0.514536
dilettantedebrah         19  0.488809
MistWeaver80             16  0.411629
Swerwin                  16  0.411629
dobbyisafreepup          15  0.385902
ohnoh18                  14  0.360175
GroundbreakingSet187     14  0.360175
Boojibs                  13  0.334448
My_Memes_Will_Cure_U     13  0.334448
thebelsnickle1991        13  0.334448
rustoo  